In [2]:
from libs.extract_features import extract_features

In [4]:
extract_features(123)

[1, 2, 3, 4]

In [6]:
from libs.predict_model import predict_model

In [8]:
predict_model(asd='asd', asd1='aad', asds='adas')

input {'asd': 'asd', 'asd1': 'aad', 'asds': 'adas'}


0.5